In [44]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim, os, pickle
import tokenize
import numpy as np
import pandas as pd

In [119]:
##### take some data
df_train = pd.read_csv('../../Data/train_clean.csv',encoding='utf-8')
qid_stack = df_train['qid1'].append(df_train['qid2'])
df_train_naExc = df_train[~df_train.isnull().any(axis=1)]
df_buf1 = df_train_naExc[['qid1','question1']]
df_buf2 = df_train_naExc[['qid2','question2']]
df_buf1.columns = ['qid','question']
df_buf2.columns = ['qid','question']
df_stack = df_buf1.append(df_buf2)
df_stack_dupExc = df_stack.drop_duplicates()
test_buf = df_stack_dupExc['question']#.head(1000)#.iloc[5000:7000]#head(1000)

In [59]:
pkl_file = open('ldamodel_traingData_5300k.pkl', 'rb')
ldamodel_dump = pickle.load(pkl_file)
pkl_file.close()
print(ldamodel_dump.keys())
##### for predict 
ldamodel = ldamodel_dump['lda_model']
dictionary = ldamodel_dump['dictionary']
##### for build the lda-model by yourself 
corpus = ldamodel_dump['encoding_corpus']

dict_keys(['tokenize_data', 'dictionary', 'encoding_corpus', 'lda_model'])


In [127]:
############# some tools
######### Tokenize the corpus
tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
def tokenize_doc(new_corpus):
    new_corpus
    texts = []
    ###
    ### Progressing bar setting
    totall_size = len(new_corpus)
    counter = 0
    progress = 0
    # loop through document list
    for i in new_corpus:    
#         ### Monitor the progress
#         if progress%(1000) ==0:
#             bar = "[" + int(progress/1000)*"|"+">"+ (10-int(progress/1000))*"-" + str((progress/100)) + "%" "]"
#             print(bar)
#         #### count
#         counter += 1
#         progress = int(10000*(counter/totall_size))

        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        
        # stem tokens
#         stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        stemmed_tokens = []
        for i in stopped_tokens:
            try:
                stemmed_tokens = stemmed_tokens + [p_stemmer.stem(i)]
            except:
                print(i)
        # add tokens to list            
        texts.append(stemmed_tokens)
    return texts

######  predict the topic
def topic_predict(new_corpus, model, dictionary):
    tokenize_buf = tokenize_doc([new_corpus])
    doc_bow = [dictionary.doc2bow(text) for text in tokenize_buf]
    que_vec = [item for itemList in doc_bow for item in itemList]
    topic_vec = ldamodel[que_vec]

    word_count_array = np.empty((len(topic_vec), 2), dtype = np.object)
    for i in range(len(topic_vec)):
        word_count_array[i, 0] = topic_vec[i][0]
        word_count_array[i, 1] = topic_vec[i][1]

    idx = np.argsort(word_count_array[:, 1])
    idx = idx[::-1]
    word_count_array = word_count_array[idx]

    final = []
    final = model.print_topic(word_count_array[0, 0], len(word_count_array))#1)

    question_topic = final.split('*') ## as format is like "probability * topic"
    return question_topic

In [128]:
###### check the topic stack
print(ldamodel.print_topics(num_topics=10, num_words=4))

[(0, '0.025*"year" + 0.023*"get" + 0.020*"engin" + 0.018*"can"'), (1, '0.056*"can" + 0.027*"use" + 0.017*"busi" + 0.016*"find"'), (2, '0.033*"like" + 0.027*"peopl" + 0.020*"feel" + 0.019*"indian"'), (3, '0.118*"best" + 0.041*"way" + 0.023*"book" + 0.019*"movi"'), (4, '0.028*"can" + 0.025*"life" + 0.017*"get" + 0.016*"first"'), (5, '0.026*"can" + 0.020*"s" + 0.015*"t" + 0.011*"world"'), (6, '0.027*"differ" + 0.017*"us" + 0.015*"s" + 0.014*"state"'), (7, '0.026*"work" + 0.024*"mean" + 0.021*"univers" + 0.015*"name"'), (8, '0.046*"make" + 0.040*"can" + 0.039*"t" + 0.028*"learn"'), (9, '0.021*"app" + 0.020*"can" + 0.015*"use" + 0.013*"human"')]


In [208]:
###### Predict a topic
pivot = 500
print(test_buf.iloc[pivot])
print(topic_predict(test_buf.iloc[pivot], ldamodel, dictionary))

How can we acquire a positive morality 
['0.028', '"can" + 0.025', '"life" + 0.017', '"get" + 0.016', '"first" + 0.015', '"quora" + 0.014', '"go" + 0.014', '"thing" + 0.013', '"someon" + 0.013', '"googl" + 0.012', '"know"']


In [210]:
###### Let's change the model by yourself
######***********************************
###### setting
num_topics = 5
display_num_topic = 5
## which data you want test
test_pivot = 1000
## How many piece
num_piece = 1000
######***********************************
######  train by a sub-set
### corpus splitting 
piece_len = int(len(corpus)/num_piece)
sub_corpus_stack = []
sub_corpus_stack = [corpus[x:x+piece_len] for x in range(0, len(corpus), piece_len)]
###
print("Splitting in " + str(num_piece)+" pieces"+", including " + str(piece_len) +" corpus per piece")

Splitting in 1000 pieces, including 537 corpus per piece


In [254]:
###### training a small LDA-model
ldamodel_self = gensim.models.ldamodel.LdaModel(sub_corpus_stack[0], 
                                                num_topics=num_topics, 
                                                id2word = dictionary, 
                                                passes=20)

In [256]:
##########################################
###### check the topic stack of your lda
##########################################
print(30*"#" + " Your LDA-Model " + 30*"#")
###### check the prediction of your lda
print("The question: '" + test_buf.iloc[test_pivot] +"'\n")
label_token = tokenize_doc([test_buf.iloc[test_pivot]])[0]
predict_stack = [topic.split("+")[0].replace('"',"").replace(" ","") for topic in topic_predict(test_buf.iloc[test_pivot], ldamodel_self, dictionary)]
figure_out_stack = [figure_out for figure_out in predict_stack if figure_out in label_token]    
print("The Token: " + str(label_token)+"\n")
print("The match topic:")
print(str(figure_out_stack))
print("The topic set: "+ str(predict_stack))
print(topic_predict(test_buf.iloc[test_pivot], ldamodel_self, dictionary))
print(90*"-")
##########################################
###### check the topic stack of huge-corpus lda as benchmark
##########################################
print("\n")
###  the large corpus
print(30*"#" + " My LDA-Model " + 30*"#")
print("The question: '" + test_buf.iloc[test_pivot] + "'\n")
predict_stack = [topic.split("+")[0].replace('"',"").replace(" ","") for topic in topic_predict(test_buf.iloc[test_pivot], ldamodel, dictionary)]
figure_out = [figure_out for figure_out in predict_stack if figure_out in label_token]    
print("The match topic:")
print(str(figure_out))
print("The topic set: "+ str(predict_stack))
print(topic_predict(test_buf.iloc[test_pivot], ldamodel, dictionary))
print(90*"-")
######

############################## Your LDA-Model ##############################
The question: 'What is the best age to teach a child how to swim '

The Token: ['best', 'age', 'teach', 'child', 'swim']

The match topic:
[]
The topic set: ['0.001', 'can', 'quora', 'look', 'life', 'porn', 'take', 'mean', 'stop', 'answer', 'question']
['0.001', '"can" + 0.001', '"quora" + 0.001', '"look" + 0.001', '"life" + 0.000', '"porn" + 0.000', '"take" + 0.000', '"mean" + 0.000', '"stop" + 0.000', '"answer" + 0.000', '"question"']
------------------------------------------------------------------------------------------


############################## My LDA-Model ##############################
The question: 'What is the best age to teach a child how to swim '

The match topic:
['best']
The topic set: ['0.118', 'best', 'way', 'book', 'movi', 'buy', 'ever', 's', 'good', 'game', 'place']
['0.118', '"best" + 0.041', '"way" + 0.023', '"book" + 0.019', '"movi" + 0.019', '"buy" + 0.018', '"ever" + 0.017', '"s

In [253]:
############ The change from number of corpus
###### Initialized LDA-model
ldamodel_self = gensim.models.ldamodel.LdaModel(sub_corpus_stack[0], 
                                                num_topics=num_topics, 
                                                id2word = dictionary, 
                                                passes=20)
### Monitor
print(15*"#" + " LDA-Model: Train Data - " + str(1) + "sub_corpus set" + 15*"#")
###### check the prediction of your lda
print("The question: '" + test_buf.iloc[test_pivot] +"'\n")
label_token = tokenize_doc([test_buf.iloc[test_pivot]])[0]
print("The Token: " + str(label_token))
##### check the prediction
predict_stack = [topic.split("+")[0].replace('"',"").replace(" ","") for topic in topic_predict(test_buf.iloc[test_pivot], ldamodel_self, dictionary)]
figure_out = [figure_out for figure_out in predict_stack if figure_out in label_token]    
print("The match topic:")
print(str(figure_out))
print("The topic set: "+ str(predict_stack))
print(topic_predict(test_buf.iloc[test_pivot], ldamodel_self, dictionary))
print(90*"-")
###### increase the corpus in model

for corpus_piece in range(1,11):
    ldamodel_self.update(sub_corpus_stack[corpus_piece])
    ### Monitor
    print(15*"#" + " LDA-Model: Train Data - " + str(corpus_piece+1) + " sub_corpus set" + 15*"#")
    ###### check the prediction of your lda
    predict_stack = [topic.split("+")[0].replace('"',"").replace(" ","") for topic in topic_predict(test_buf.iloc[test_pivot], ldamodel_self, dictionary)]
    figure_out = [figure_out for figure_out in predict_stack if figure_out in label_token]    
    print("The match topic:")
    print(str(figure_out))
    print("The topic set: "+ str(predict_stack))
    print(topic_predict(test_buf.iloc[test_pivot], ldamodel_self, dictionary))
    print(90*"-")

### Monitor
print(15*"#" + " LDA-Model: Train Data - " + str(num_piece) + " sub_corpus set" + 15*"#")
predict_stack = [topic.split("+")[0].replace('"',"").replace(" ","") for topic in topic_predict(test_buf.iloc[test_pivot], ldamodel, dictionary)]
figure_out = [figure_out for figure_out in predict_stack if figure_out in label_token]    
print("The match topic:")
print(str(figure_out))
print("The topic set: "+ str(predict_stack))
print(topic_predict(test_buf.iloc[test_pivot], ldamodel, dictionary))
print(90*"-")

############### LDA-Model: Train Data - 1sub_corpus set###############
The question: 'What is the best age to teach a child how to swim '

The Token: ['best', 'age', 'teach', 'child', 'swim']
The match topic:
[]
The topic set: ['0.001', 'can', 'use', 'will', 'take', 't', 'war', 'love', 'find', 'differ', 'mani']
['0.001', '"can" + 0.001', '"use" + 0.001', '"will" + 0.000', '"take" + 0.000', '"t" + 0.000', '"war" + 0.000', '"love" + 0.000', '"find" + 0.000', '"differ" + 0.000', '"mani"']
------------------------------------------------------------------------------------------
############### LDA-Model: Train Data - 2 sub_corpus set###############
The match topic:
[]
The topic set: ['0.002', 'math', 'relationship', 'develop', 'use', 'bodi', 'mobil', 'believ', 'hard', 'comput', 'ever']
['0.002', '"math" + 0.001', '"relationship" + 0.001', '"develop" + 0.001', '"use" + 0.001', '"bodi" + 0.001', '"mobil" + 0.001', '"believ" + 0.001', '"hard" + 0.001', '"comput" + 0.001', '"ever"']
---------